# Object Detection with Classification & Localization

## 1. Importing Libraries

In [1]:
# Libs
import numpy as np
import matplotlib.pyplot as plt
# image processing
import PIL.Image , PIL.ImageFont , PIL.ImageDraw

In [9]:
import os

In [2]:
# Tensorflow Utils
import tensorflow as tf
import tensorflow_datasets as tfds

## 2. Visualization Utilities

In [3]:
# Image properties
im_widht = 75
im_height = 75
use_normalized_coordinates = True

In [4]:
# Bounding Boxes
def draw_bounding_box_on_image(image , ymin , xmin , ymax , xmax , color = 'red' , thickness = 1 , use_normalized_coordinates = True):
  draw = PIL.ImageDraw.Draw(image) # creating draw object
  im_width , im_height = image.size # image width & height

  if use_normalized_coordinates:
    (left , right , top , bottom ) = (xmin * im_width , xmax * im_width , ymin * im_height , ymax * im_height)
  else:
    (left , right , top , bottom ) = (xmin , xmax , ymin , ymax)

  # drawing bounding box lines
  draw.line(xy = [(left , top) , (left , bottom) ,
                    (right , bottom) , (right , top) , (left , top)],
            width = thickness ,  fill = color)


def draw_bounding_boxes_on_image(image , boxes , color = [] , thickness = 1 , display_str_list = ()):
  boxes_shape = boxes.shape

  # if None
  if not boxes_shape:
    return None
  # If incorrect shape
  if len(boxes_shape)!= 2 or boxes_shape[-1] != 4:
    raise ValueError("Input size must be [N , 4]")
  # looping through all objects
  for i in range(boxes_shape[0]):
    draw_bounding_box_on_image(image ,
                               boxes[i , 1] ,
                               boxes[i , 0] ,
                               boxes[i , 3] ,
                               boxes[i , 2] ,
                               color[i] ,
                               thickness ,
                               display_str_list[i]
                               )

# Drawing bounding on numpy array image
def draw_bounding_box_on_image_array(image , boxes , color = [] , thickness = 1 , display_str_list = ()):
  image_pil = PIL.Image.fromarray(image)

  rgb_image = PIL.Image.new("RGBA" , image_pil.size)
  rgb_image.paste(image_pil)

  draw_bounding_boxes_on_image(rgb_image , boxes , color = [] , thickness =  1 , display_str_list = ())
  return np.array(rgb_image)



In [8]:
def dataset_to_numpy_util(training_dataset , validation_dataset , N):
  batch_train_ds = training_dataset.unbatch().batch(N)

  # if it
  if tf.executing_eagerly():

    # Training Dataset
    for training_digits , (training_labels , training_bboxes) in training_dataset :
      training_digits = training_digits.numpy()
      validation_labels = training_labels.numpy()
      training_bboxes = training_bboxes.numpy()

    # Validation Dataset
    for validation_digits , (validation_labels , validation_bboxes) in validation_dataset :
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      validation_bboxes = validation_bboxes.numpy()


  return (training_digits , training_labels , training_bboxes ,
          validation_digits , validation_labels , validation_bboxes)


In [11]:
# Generating synthetic digit images from fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")

# n - number of images to be created
def create_digits_from_local_fonts(n):
  # font
  font_labels = []
  img = plt.Image.new("LA" , ( 75, 75 * n ) , color =  (0 , 255))
  font1 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'DejaVuSansMono-Oblique.ttf'), 25)
  font2 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'STIXGeneral.ttf'), 25)

  # Draw images
  draw = PIL.ImageDraw.Draw(img)
  for i in range(n):
      font_labels.append(i % 10)
      draw.text((7 + i*75, 0 if i < 10 else -4),
            str(i % 10),
            fill=(255,255),
            font=font1 if i < 10 else font2)

  font_digits = np.array(img.getdata(), np.float32)[:,0] / 255.0


  font_digits = np.reshape(
    np.stack(
        np.split(
            np.reshape(font_digits, [75, 75*n]),
            n, axis=1
        ),
        axis=0
    ),
    [n, 75*75]
  )


  return font_digits , font_labels


In [12]:
# For displaying digits with boundingboxes
def display_digits_with_boxes(digits, predictions, labels, pred_bboxes, bboxes, iou, title):
    n = 10

    indexes = np.random.choice(len(predictions), size=n)
    n_digits = digits[indexes]
    n_predictions = predictions[indexes]
    n_labels = labels[indexes]

    n_iou = []
    if len(iou) > 0:
        n_iou = iou[indexes]

    if len(pred_bboxes) > 0:
        n_pred_bboxes = pred_bboxes[indexes]

    if len(bboxes) > 0:
        n_bboxes = bboxes[indexes]

    n_digits = n_digits * 255.0
    n_digits = n_digits.reshape(n, 75, 75)

    fig = plt.figure(figsize=(20, 4))
    plt.title(title)
    plt.xticks([])
    plt.yticks([])

    for i in range(10):
        ax = fig.add_subplot(1, 10, i + 1)
        bboxes_to_plot = []

        if (len(pred_bboxes) > i):
            boxes_to_plot.append(n_pred_bboxes[i])

        if (len(bboxes) > i):
            boxes_to_plot.append(n_bboxes[i])

        img_to_draw = draw_bounding_boxes_on_image_array(
            image=n_digits[i],
            boxes=np.asarray(bboxes_to_plot),
            color=['red', 'green'],
            display_str_list=['True', 'Pred']
        )

        plt.xlabel(n_predictions[i])
        plt.xticks([])
        plt.yticks([])

        if n_predictions[i] != n_labels[i]:
            ax.xaxis.label.set_color('red')

        plt.imshow(img_to_draw)

        if len(iou) > i:
            color = "black"
            if (n_iou[i][0] < iou_threshold):
                color = "red"
            ax.text(0.2, -0.3, "iou: %s" % (n_iou[i][0]), color=color, transform=ax.transAxes)
